# <B> Clean up </B>

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## clean

In [2]:
import os
import time
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [4]:
strBucketName = pm.get_params(key=strPrefix + "BUCKET")

In [5]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

remove_bucket failed: s3://sagemaker-us-east-1-419974056037 An error occurred (AccessDenied) when calling the DeleteBucket operation: Access Denied


* endpoint

In [6]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [7]:
clean = clean_up()
sm_client = boto3.client('sagemaker')
strEndPointName = pm.get_params(key=strPrefix + "ENDPOINT-NAME")
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

--- Deleted model: pytorch-inference-2023-04-06-01-06-42-638
--- Deleted endpoint: endpoint-model-2303041680741691
--- Deleted endpoint_config: endpoint-model-2303041680741691


* pipeline

In [8]:
strPipelineName = pm.get_params(key=strPrefix + "PIPELINE-NAME")
strPipelineName

'SM-MLOPS-PIPELINE-PIPELINE-TRAIN-model-230304'

In [9]:
!aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:us-east-1:419974056037:pipeline/sm-mlops-pipeline-pipeline-train-model-230304"
}


* model registry

In [10]:
sagemaker_client = boto3.client('sagemaker')

In [11]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [12]:
DeleteModelGroupName = pm.get_params(key=strPrefix + "MODEL-GROUP-NAME")
#DeleteModelGroupName = "model-24"
empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

* parameter store

In [17]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]

total_size = len(listParams)
for idx in range(0, total_size, 10):
    start = idx
    if start + 10 < total_size: end = start + 10
    else: end = total_size
    listSubParams = listParams[start:end]
    pm.delete_param(listSubParams)

['SM-MLOPS-PIPELINE-ACCOUNT-ID',
 'SM-MLOPS-PIPELINE-BUCKET',
 'SM-MLOPS-PIPELINE-CODE-BUILD-ROLE-ARN',
 'SM-MLOPS-PIPELINE-CODE-PIPELINE-ROLE-ARN',
 'SM-MLOPS-PIPELINE-DATA-PATH',
 'SM-MLOPS-PIPELINE-ENDPOINT-NAME',
 'SM-MLOPS-PIPELINE-LAMBDA-ROLE-ARN',
 'SM-MLOPS-PIPELINE-MODEL-GROUP-NAME',
 'SM-MLOPS-PIPELINE-PIPELINE-NAME',
 'SM-MLOPS-PIPELINE-REGION',
 'SM-MLOPS-PIPELINE-SAGEMAKER-ROLE-ARN',
 'PREFIX']

* code commit, buldand pipeline

In [14]:
from utils.code_pipeline import code_pipeline_handler
cph = code_pipeline_handler()

In [15]:
cph.delete_repository(strRepoName="SM-MLOPS-PIPELINE-MLOPS")
cph.delete_build_project(strCodeBuildPJTName="demo-mlops-build")
cph.delete_pipeline(strPipeLineName="demo-mlops-code-pipeline")

== DELETE REPO ==
  Repository name [SM-MLOPS-PIPELINE-MLOPS] was successfully deleted!!
''
''
